### Задание 1

In [171]:
class Stack:
    
    def __init__(self):
        self.stack = []
        
    def push(self, element):
        self.stack.append(element)
        
    def pop(self):
        if self.stack:
            return self.stack.pop()
        else:
            raise Exception('Stack is empty')

    def __str__(self):
        return "; ".join(self.stack)

            
class TaskManager:
    
    def __init__(self):
        self.tasks = {}
    
    def new_task(self, action, priority):
        if priority not in self.tasks:
            self.tasks[priority] = Stack()
        self.tasks[priority].push(action)
        
    def __str__(self):
        ans = [f"{k} {v}" for k, v in sorted(self.tasks.items())]
        return "\n".join(ans)

In [172]:
manager = TaskManager()
manager.new_task("сделать уборку", 4)
manager.new_task("помыть посуду", 4)
manager.new_task("отдохнуть", 1)
manager.new_task("поесть", 2)
manager.new_task("сдать дз", 2)
print(manager)

1 отдохнуть
2 поесть; сдать дз
4 сделать уборку; помыть посуду


### Задание 2

In [2]:
from collections import OrderedDict

In [4]:
class LRUCache:
    def __init__(self, capacity):
        self.capacity = capacity
        self.dict = OrderedDict()
        
    def get(self, key):
        self.dict.move_to_end(key)
        return self.dict[key]
    
    @property
    def cache(self):
        return next(iter(self.dict))
    
    @cache.setter
    def cache(self, new_elem):
        key, value = new_elem
        if len(self.dict) == self.capacity:
            self.dict.popitem(last=False)
        self.dict[key] = value
    
    def print_cache(self):
        print("LRU cache:")
        for k, v in self.dict.items():
            print(f"{k} : {v}")

In [5]:
cache = LRUCache(3)
cache.cache = ("key1", "value1")
cache.cache = ("key2", "value2")
cache.cache = ("key3", "value3")

In [6]:
cache.print_cache()

LRU cache:
key1 : value1
key2 : value2
key3 : value3


In [7]:
print(cache.get("key2"))

value2


In [8]:
cache.cache = ("key4", "value4")

In [9]:
cache.print_cache()

LRU cache:
key3 : value3
key2 : value2
key4 : value4


### Задание 3

In [17]:
def decor(func):
    
    def wrapper(*args, **kwargs):
        print('csdc')
        a = func(*args, **kwargs)
        print('cedw')
        return a
    return wrapper

In [18]:
@decor
def sum_digs(a, b):
    return a + b

In [78]:
class LRUCache:
    def __init__(self, capacity):
        self.capacity = capacity
        self.dict = OrderedDict()
        
    def __call__(self, func):
        
        def wrapper(*args, **kwargs):
            args_list = list(args)
            #Сортируем, чтобы не зависеть от порядка ввода именованных аргументов
            kwargs_list = sorted(kwargs.items())
            args_list.extend(kwargs_list)
            all_args = tuple(args_list)
            if all_args in self.dict:
                return self.dict[all_args]
            value = func(*args, **kwargs)
            self._update_dict(all_args, value)
            return value
        
        return wrapper
    
    def _update_dict(self, all_args, value):
        if len(self.dict) == self.capacity:
            self.dict.popitem(last=False)
        self.dict[all_args] = value

In [79]:
@LRUCache(10000)
def fibonaci(n):
    if n <= 1:
        return n
    else:
        return(fibonaci(n-1) + fibonaci(n-2))

In [80]:
fibonaci(1000)

43466557686937456435688527675040625802564660517371780402481729089536555417949051890403879840079255169295922593080322634775209689623239873322471161642996440906533187938298969649928516003704476137795166849228875

### Задание 4

In [93]:
from dataclasses import dataclass
from typing import Set

In [103]:
{1, 2, 4}.issubset({3, 2, 4})

False

In [168]:
WINNING_SEQUENCES = ({0, 1, 2}, {3, 4, 5}, {6, 7, 8}, {0, 3, 6},
                   {1, 4, 7}, {2, 5, 8}, {0, 4, 8}, {2, 4, 6})

class Cell:
    def __init__(self, number):
        self.number = number
        self.value = None
        self.player = None
    
    def is_empty(self):
        return self.value is None
    
    def fill(self, value, player):
        assert value in ['X', 'O'], "Value should be 'X' or 'O'"
        assert self.is_empty(), "This cell is not empty"
        self.value = value
        self.player = player

    

class Player:
    def __init__(self, name):
        self.name = name
        self.cross_positions = set()
        self.zero_positions = set()
    
    def add_turn(self, value, number):
        assert value in ['X', 'O'], "Value should be 'X' or 'O'"
        if value == 'X':
            self.cross_positions.add(number)
        else:
            self.zero_positions.add(number)
        
class Game:
    def __init__(self, first_player, second_player, first_player_starts = True):
        self.board = [Cell(i) for i in range(9)]
        self.first_player = first_player
        self.second_player = second_player
        self.is_first_player_turn = first_player_starts
        self.game_over = False
        self.winner = None
    
    def _make_move(self, number, value, player):
        self.board[number].fill(value, player)
        player.add_turn(value, number)
        
    def _next_turn(self):
        self.is_first_player_turn = not self.is_first_player_turn
    
    def first_player_move(self, value, number):
        self._is_game_over()
        if self.is_first_player_turn:
            self._make_move(number, value, self.first_player)
            self._check_result(self.first_player)
            self._next_turn()
        else:
            raise Exception(f"'It's the second player's ({self.second_player.name}) turn")
    
    def second_player_move(self, value, number):
        self._is_game_over()
        if not self.is_first_player_turn:
            self._make_move(number, value, self.second_player)
            self._check_result(self.first_player)
            self._next_turn()
        else:
            raise Exception(f"It's the first player's ({self.first_player.name}) turn")
    
    def _is_player_win(self, player):
        cross = player.cross_positions
        zero = player.zero_positions
        for win_sequence in WINNING_SEQUENCES:
            if win_sequence.issubset(zero) or win_sequence.issubset(cross):
                return True
        return False
    
    def _check_draw(self):
        for cell in self.board:
            if cell.is_empty():
                return False
        return True
    
    def _check_result(self, player):
        if self._is_player_win(player):
            print(f"{player.name} is win")
            self.winner = player
            self.game_over = True
        if self._check_draw():
            print("It's a draw")
            self.game_over = True
    
    def _is_game_over(self):
        if self.game_over:
            if self.winner:
                raise Exception(f"Game over. {self.winner.name} won")
            else:
                raise Exception(f"Game over. It's a draw")

In [169]:
Ivan = Player('Ivan')
Vasya = Player('Vasya')
game = Game(Ivan, Vasya)

In [170]:
game.first_player_move('X', 3)
game.second_player_move('O', 0)
game.first_player_move('X', 4)
game.second_player_move('O', 1)
game.first_player_move('X', 5)
game.second_player_move('O', 6)

Ivan is win


Exception: Game over. Ivan won

In [ ]:
game.first_player_move('X', 3)
game.second_player_move('O', 0)
game.first_player_move('X', 4)
game.second_player_move('O', 1)
game.first_player_move('X', 5)
game.second_player_move('O', 6)